# General Image Inferencing

This is a demonstration of a generalized framework to get inferences from classification, object detection, and semantic segmentation models using TensorFlow Serving. This work is heavily based on [the object_detection section of the TensorFlow Models repository](https://github.com/tensorflow/models/tree/master/research/object_detection). 

## Model Specification

This notebook is a demonstration of a single system which could be used for inferencing on any number of classification, object detection, or semantic segmentation models. To make this possible, the model being served has to conform to a specification.

### Input

let `n_img` := 1. TODO: Allow processing of more than one image at a time?

- `inputs`, type: `uint8`, shape: `[n_img, -1, -1, 3]`. This is the image. 

### Output

- `num_detections`, type: `int`, shape: `[n_img]`. The value of this field is always less than or equal to `len_detections`.
- `detection_classes`, type: `int`, shape: `[n_img, len_detections]`. Class identifier. 
- `detection_scores`, type: `float`, shape: `[n_img, len_detections]`. Confidence level.
- `detection_boxes`, type: `float`, shape: `[n_img, len_detections, 4]`. Values are on `[0,1)`, representing the bounds of the boxes. (object detection & semantic segmentation only)
- `detection_masks`, type: `int`, shape: `[n_img, len_detections, -1, -1]`. Value is either `1` or `0`, meaning to be described. (semantic segmentation only)

## Serving the model

Do something like:

```sudo docker run -it -p 8500:8500 --mount type=bind,source=<path to model>,target=/models/<model name> -e MODEL_NAME=<model name> -t tensorflow/serving```

In [1]:
import grpc
import tensorflow as tf
from PIL import Image
import numpy as np

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

import label_map_util

## Configuring this Notebook

`SERVER_ADDR`: The address of the TensorFlow Serving server.  
`MODEL_SPEC_NAME`: The name of the model, which you specified when you started the TensorFlow Serving server.  
`IMAGE_PATH`: The path to the image you want to inference on.  
`LABEL_MAP_PATH`: The path to a text-format protobuf which provides mapping from the integer class identifiers your model uses to their textual descriptions.  

In [2]:
SERVER_ADDR = 'localhost:8500'
MODEL_SPEC_NAME = '23'
IMAGE_PATH = './brompton.jpg'
LABEL_MAP_PATH = 'data/mscoco_label_map.pbtxt'

In [3]:
im = Image.open(IMAGE_PATH)
numpy_ary = np.array([np.array(im, dtype=np.uint8)])

In [4]:
channel = grpc.insecure_channel(SERVER_ADDR)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [5]:
request = predict_pb2.PredictRequest()
request.model_spec.name = MODEL_SPEC_NAME
request.inputs['inputs'].CopyFrom(tf.contrib.util.make_tensor_proto(numpy_ary))

In [6]:
result = stub.Predict(request, 10.0)

In [7]:
label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

In [8]:
num_detections = int(result.outputs['num_detections'].float_val[0])
classes = result.outputs['detection_classes'].float_val[:num_detections]
scores = result.outputs['detection_scores'].float_val[:num_detections]

for i in range(num_detections):
    if scores[i] > 0.10:
        print(label_map[int(classes[i])]['name'], scores[i])

person 0.963925838470459
person 0.9449862241744995
bicycle 0.9167641997337341
person 0.8627572655677795
handbag 0.654525876045227
person 0.6309804916381836
car 0.6057604551315308
car 0.595432698726654
car 0.5520927309989929
bicycle 0.4168332815170288
bicycle 0.357604056596756
car 0.3359058201313019
truck 0.32337236404418945
person 0.2555420994758606
handbag 0.22566813230514526
person 0.2144150286912918
bicycle 0.21365347504615784
bicycle 0.17084233462810516
skateboard 0.16359734535217285
bicycle 0.15253078937530518
person 0.12370678782463074
bicycle 0.12173733115196228
bicycle 0.11699391901493073
bicycle 0.11693298816680908
bicycle 0.11648998409509659
handbag 0.11604130268096924
bicycle 0.1125880554318428
motorcycle 0.11031841486692429
backpack 0.10968058556318283
bicycle 0.1078219935297966
car 0.10715402662754059
traffic light 0.10337898135185242
bicycle 0.10292237251996994
bicycle 0.10136948525905609
